<a href="https://colab.research.google.com/github/Riazhatvi/-AI-Text-Based-Mental-Health-Support-System/blob/main/Ai_MHS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
from nltk.tokenize import word_tokenize
nltk.download('all')

#we use this to download english library
spacy.cli.download("en_core_web_sm")

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
# The First step is Loading Data in json file
import json

file_path = '/content/drive/MyDrive/Mental_Health_Project/Mental_health.json'

with open(file_path, 'r') as file:
    data = json.load(file)
type(data)

dict

In [ ]:
# Now its time to tokenize corpus
from nltk.tokenize import word_tokenize
# nltk.download('punkt')
if not isinstance(data, str):
    data = str(data)
tokens = word_tokenize(data)

print(tokens)


['{', "'intents", "'", ':', '[', '{', "'tag", "'", ':', "'greeting", "'", ',', "'patterns", "'", ':', '[', "'Hi", "'", ',', "'Hey", "'", ',', "'Is", 'anyone', 'there', '?', "'", ',', "'Hi", 'there', "'", ',', "'Hello", "'", ',', "'Hey", 'there', "'", ',', "'Howdy", "'", ',', "'Hola", "'", ',', "'Bonjour", "'", ',', "'Konnichiwa", "'", ',', "'Guten", 'tag', "'", ',', "'Ola", "'", ']', ',', "'responses", "'", ':', '[', "'Hello", 'there', '.', 'Tell', 'me', 'how', 'are', 'you', 'feeling', 'today', '?', "'", ',', "'Hi", 'there', '.', 'What', 'brings', 'you', 'here', 'today', '?', "'", ',', "'Hi", 'there', '.', 'How', 'are', 'you', 'feeling', 'today', '?', "'", ',', "'Great", 'to', 'see', 'you', '.', 'How', 'do', 'you', 'feel', 'currently', '?', "'", ',', '``', 'Hello', 'there', '.', 'Glad', 'to', 'see', 'you', "'re", 'back', '.', 'What', "'s", 'going', 'on', 'in', 'your', 'world', 'right', 'now', '?', '``', ']', '}', ',', '{', "'tag", "'", ':', "'morning", "'", ',', "'patterns", "'", ':', 

In [ ]:
# Now time to apply stop words and lower the capitilization
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
filtered_text = [word for word in tokens if word.lower() not in stop_words]
clean_sentence = ' '.join(filtered_text)
print(clean_sentence)

{ 'intents ' : [ { 'tag ' : 'greeting ' , 'patterns ' : [ 'Hi ' , 'Hey ' , 'Is anyone ? ' , 'Hi ' , 'Hello ' , 'Hey ' , 'Howdy ' , 'Hola ' , 'Bonjour ' , 'Konnichiwa ' , 'Guten tag ' , 'Ola ' ] , 'responses ' : [ 'Hello . Tell feeling today ? ' , 'Hi . brings today ? ' , 'Hi . feeling today ? ' , 'Great see . feel currently ? ' , `` Hello . Glad see 're back . 's going world right ? `` ] } , { 'tag ' : 'morning ' , 'patterns ' : [ 'Good morning ' ] , 'responses ' : [ `` Good morning . hope good night 's sleep . feeling today ? `` ] } , { 'tag ' : 'afternoon ' , 'patterns ' : [ 'Good afternoon ' ] , 'responses ' : [ 'Good afternoon . day going ? ' ] } , { 'tag ' : 'evening ' , 'patterns ' : [ 'Good evening ' ] , 'responses ' : [ 'Good evening . day ? ' ] } , { 'tag ' : 'night ' , 'patterns ' : [ 'Good night ' ] , 'responses ' : [ 'Good night . Get proper sleep ' , 'Good night . Sweet dreams . ' ] } , { 'tag ' : 'goodbye ' , 'patterns ' : [ 'Bye ' , 'See later ' , 'Goodbye ' , 'Au revoir

In [ ]:
#  Now we will Use BoW for basic text classification and initial filtering
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform([clean_sentence])
bow_array = X.toarray()
feature_names = vectorizer.get_feature_names_out()
bow_df = pd.DataFrame(bow_array, columns=feature_names)
print(bow_df)


   10  11  12  13  14  15  16  17  18  19  ...  would  writing  written  \
0   2   2   2   4   2   4   2   2   8   2  ...     28        1        2   

   wrong  yeah  years  yes  you  young  younger  
0      6     6      2    6   15      6        2  

[1 rows x 1167 columns]


In [ ]:
# Apply lemmatization to the preprocessed text in bag of words
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
lemmatized_words = [lemmatizer.lemmatize(word) for word in bow_df]
print(lemmatized_words)

['10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '70', '75', '90', '9152987821', 'ability', 'ability', 'able', 'about', 'absolutely', 'abuse', 'access', 'according', 'ache', 'achievable', 'achieve', 'achieved', 'acknowledging', 'act', 'actionable', 'activity', 'activity', 'actual', 'actually', 'addition', 'additional', 'additionally', 'adolescence', 'adult', 'advice', 'advise', 'affect', 'affected', 'affect', 'affirmation', 'afraid', 'afternoon', 'age', 'agent', 'age', 'aggressive', 'agree', 'ai', 'aid', 'aim', 'alcohol', 'alleviate', 'alone', 'along', 'alot', 'already', 'alright', 'also', 'alternative', 'although', 'always', 'am', 'america', 'and', 'anger', 'angry', 'another', 'answer', 'answered', 'anxiety', 'anxious', 'any', 'anymore', 'anyone', 'anything', 'anyway', 'appears', 'appetite', 'applied', 'approach', 'approach', 'approaching', 'appropriate', 'are', 'area', 'area', 'arise', 'around',

In [ ]:
# encoding text
from gensim.models import Word2Vec
model = Word2Vec([lemmatized_words], vector_size=100, window=10, min_count=1, sg=0)
embeddings = [model.wv[word] for word in lemmatized_words if word in model.wv.key_to_index]
embeddings

[array([ 3.59285600e-03,  8.85754637e-03,  5.36554074e-03, -9.67724621e-03,
         9.66733205e-05,  3.92637728e-03,  3.40222265e-03, -1.47135730e-03,
        -2.50446517e-03, -4.15121205e-03,  3.21688293e-03,  2.57693697e-03,
         8.69730767e-03, -7.99040496e-03, -2.65948847e-03,  1.82158814e-03,
         9.92295984e-03,  2.95076054e-03, -4.07222472e-03,  4.65726666e-03,
        -6.65482786e-03, -1.39369816e-03, -6.80629490e-03, -6.56624045e-03,
         1.19964476e-03,  2.43886071e-03,  4.30673501e-03,  6.29898021e-03,
        -7.46904220e-03, -2.96951085e-03, -4.77559585e-03,  1.99318180e-04,
         4.26632864e-03, -3.23035358e-03,  5.99948503e-03,  8.65436066e-03,
         4.08598455e-03,  6.62126625e-03, -6.25346415e-03, -2.89977249e-03,
        -5.17707877e-03, -6.33659959e-03,  8.15536175e-03, -3.52072483e-03,
        -7.91602954e-03, -3.85925616e-03, -8.35310202e-03, -1.44145743e-03,
         6.40070578e-03, -2.80036801e-03, -1.40594109e-03, -6.46110252e-03,
        -1.1